In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
import os
import random
import shutil

# Specify the root folder containing the dataset
mask_root = ' path_to_dataset_folder '
mask_path = 'mask_detection/'
label_ = '.txt'
img_ = '.jpg'

# Retrieve file list of annotations
file_list = [file for file in os.listdir(mask_root) if file.endswith(".txt")]

# Create directory structure
folder_list = ['mask_detection', 'mask_detection/train', 'mask_detection/val', 'mask_detection/train/images', \
                'mask_detection/train/labels', 'mask_detection/val/images', 'mask_detection/val/labels']
for folder in folder_list:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
# Shuffle and split the dataset into training and validation sets
random.shuffle(file_list)
test_ratio = 0.1
test_list = file_list[:int(len(file_list)*test_ratio)]
train_list = file_list[int(len(file_list)*test_ratio):]

# Copy files to respective directories for training and validation sets
for i in test_list:
    f_name = os.path.splitext(i)[0]
    shutil.copyfile(os.path.join(mask_root,(f_name+img_)), os.path.join(mask_path, 'val/images', (f_name+img_)))
    shutil.copyfile(os.path.join(mask_root,(f_name+label_)), os.path.join(mask_path, 'val/labels', (f_name+label_)))
for i in train_list:
    f_name = os.path.splitext(i)[0]
    shutil.copyfile(os.path.join(mask_root,(f_name+img_)), os.path.join(mask_path, 'train/images', (f_name+img_)))
    shutil.copyfile(os.path.join(mask_root, (f_name+label_)), os.path.join(mask_path, 'train/labels', (f_name+label_)))

In [ ]:
# Create and save configuration file in YAML format
import yaml
data =dict()

data['train'] = 'path_to_mask_detection_folder + mask_detection/train'
data['val'] = 'path_to_mask_detection_folder + mask_detection/val'
data['test'] = 'path_to_mask_detection_folder + mask_detection/val'

data['nc'] = 3 # Number of classes
data['names'] =['OK','improperly', 'NO'] # Class names

with open('mask_detection.yaml', 'w') as f:
	yaml.dump(data, f)

In [ ]:
# Train the model
from ultralytics import YOLO
model = YOLO('yolov8s.yaml')
results = model.train(data ='mask_detection.yaml', epochs = 100, batch=32,device = 0  , patience=30, name='road_sign_s')

In [ ]:
from ultralytics import YOLO

# Load the trained model
model_path = 'path_to_trained_weights'
model = YOLO(model_path)  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered

# Validate the model
print("map50-95", metrics.box.map)
print("map50", metrics.box.map50)

In [ ]:
# Perform inference on test images
from ultralytics import YOLO
import cv2
import os
from ultralytics.yolo.utils.plotting import Annotator
import matplotlib.pyplot as plt
import numpy as np

model = YOLO('weights_file_path')

root_folder = 'path_to_test_images_folder'
result_folder = '../dataset/mask_detection/result'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

test_img_list = os.listdir(root_folder)
device = 'cpu'
color_dict = [(0, 255, 0), (255, 0, 0), (0, 0, 255)]

In [ ]:
# Read and process each test image
test_img = cv2.imread(os.path.join(root_folder, test_img_list[0]))
img_src = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
results = model(test_img)
# Annotate the image with bounding boxes and labels
for result in results:
    annotator = Annotator(img_src)
    boxes = result.boxes
    for box in boxes:
        b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
        cls = box.cls
        cls = box.cls
        annotator.box_label(b, model.names[int(cls)], color_dict[int(cls)])
img_src = annotator.result()
# Display annotated image
plt.imshow(img_src)
plt.show()